In [48]:
import recordlinkage
from Annotation_helper import *
import numpy as np

In [50]:
transactions = load_transactions("../../Data/structured_data\VOC Cochin Slave Transactions 1706-1801 - IISH - Database - 2018 - v1.csv")
permissions = load_permissions("../../Data/structured_data\VOC Cochin Slave Transport Permissions 1770-1795 - IISH - Database - 2018 - v1.csv")

In [44]:
indexer = recordlinkage.Index()
indexer.full()
candidate_links = indexer.index(permissions, transactions)

compare_cl = recordlinkage.Compare()

compare_cl.exact('SlaafGender', 'SlaafGender', label='SlaafGender')
compare_cl.string('SlaafNaamNieuw', 'SlaafNaamNieuw', label='SlaafNaamNieuw')
compare_cl.exact('BezitterGender', 'KoperGender', label='BezitterGender')
compare_cl.string('BezitterVoornaam', 'KoperVoornaam', label='BezitterVoornaam')
compare_cl.string('BezitterBeroep', 'KoperBeroep', label='BezitterBeroep')
compare_cl.string('BezitterAchternaam', 'KoperAchternaam', label='BezitterAchternaam')
n_batches = 100
split_perm = np.array_split(permissions, n_batches)

In [24]:
for i, p in enumerate(split_perm):
    print(f"[{i}/{n_batches}]")
    candidate_links = indexer.index(p, transactions)
    features = compare_cl.compute(candidate_links, p, transactions)
    features = features.reset_index()
    features = features.rename(columns={"level_0": "permission_indx", "level_1": "transaction_indx"})
    features.to_csv(f"Distances/distances_{i}.csv", index=False)
    clear_output(wait=True)

[0/100]
      permission_indx  transaction_indx  SlaafGender  SlaafNaamNieuw  \
0                 208                 0            1        0.000000   
1                 208                 1            1        0.250000   
2                 208                 2            0        0.125000   
3                 208                 3            1        0.333333   
4                 208                 4            1        0.750000   
...               ...               ...          ...             ...   
7183              208              7192            0        0.125000   
7184              208              7193            1        0.000000   
7185              208              7194            1        0.125000   
7186              208              7195            1        0.111111   
7187              208              7196            0        0.250000   

      BezitterGender  BezitterVoornaam  BezitterBeroep  BezitterAchternaam  
0                  1             0.250        0.16